## Creating a Universal Reference Compressor
There exist two approaches here
1. Linearly assign $(n^2)$
2. Dynamic programming to assign absolute optimal compression

### Linear Assignment

In [ ]:
def get_tree(d, keys):
    for k in keys:
        d = d[k]
    return d

def set_tree(d, keys, value):
    d = get_tree(d, keys)
    d[value] = {}
    
def get_indices(d):
    for k in d.keys():
        if type(k) == tuple:
            return k
    return ()

In [ ]:
def build_tree(reference):
    tree = {}
    # Convert reference into tree
    for i in range(len(reference)):
        if (i != 0 and i % 10000 == 0):
            print(i)
        for j in range(i, len(reference)):
            parents = [reference[x] for x in range(j-i,j)]
            set_tree(tree, parents, reference[j])
            set_tree(tree, parents + [reference[j]], (j-len(parents),j))
    return tree
def linear_ref_compress(reference, compress, tree):        
    compressed_ref = []
    temp_tree = tree
    for i in range(len(compress)):
        #print(compress[i])
        #print(temp_tree.keys())
        if compress[i] in temp_tree:
            temp_tree = temp_tree[compress[i]]
            if len(temp_tree.keys()) > 1:
                next
            else:
                #print(get_indices(temp_tree))
                compressed_ref.append(get_indices(temp_tree))
                temp_tree = tree
        else:
            #print(get_indices(temp_tree))
            compressed_ref.append(get_indices(temp_tree))
            temp_tree = tree[compress[i]]
    final_index = get_indices(temp_tree)
    if final_index != ():
        compressed_ref.append(final_index)
    return compressed_ref

In [ ]:
reference = "ACGTACTGACTG"
compress = "ACGTACTGACTG"
linear_ref_compress(reference, compress, build_tree(reference))

In [ ]:
def binarize(char):
    char_dict = {"A": "000", "C": "001", "T":"010", "G":"011", "N":"100", "-":"101", "_ref": "111"}
    return char_dict[char]
def compress_regular(genome):
    out_genome = ""
    for g in genome:
        out_genome += binarize(g)
    return out_genome
def linear_ref_compress_genome(reference):
    tree = build_tree(reference)
    def compress_genome(genome):
        char_dict = {"A": "000", "C": "001", "T":"010", "G":"011", "N":"100", "-":"101", "_ref": "111"}
        ref_indices = linear_ref_compress(reference, genome, tree)
        compressed = ""
        for (start, end) in ref_indices:
            if (end-start) > 6:
                compressed += binarize("_ref")
                compressed += str('{:08b}'.format(start))
                compressed += str('{:08b}'.format(end))
            else:
                for x in range(start, end+1):
                    compressed += binarize(reference[x])
        return compressed
    return compress_genome
def linear_ref_decompress_genome(reference):
    def decompress_genome(compressed):
        char_dict = {"A": "000", "C": "001", "T":"010", "G":"011", "N":"100", "-":"101", "_ref": "111"}
        rev_dict = {char_dict[k]:k for k in char_dict.keys()}
        i = 0
        decompressed = ""
        while i < len(compressed):
            tag = rev_dict[compressed[i:i+3]]
            if tag == "_ref":
                i += 3
                start = int(compressed[i:i+8], 2)
                i += 8
                end = int(compressed[i:i+8], 2)
                i += 8
                decompressed += reference[start:end+1]
            else:
                i += 3
                decompressed += tag
        return decompressed
    return decompress_genome

In [ ]:
reference = "ACGATCGACTGACTGACTGACTAGCT"
compress = "ACGTACGATCGACTAGCTACGATCG"
compress_func = linear_ref_compress_genome(reference)
decompress_func = linear_ref_decompress_genome(reference)
assert(decompress_func(compress_func(compress)) == compress)

In [ ]:
from data_process import *

## Testing Random Reference
Now, let's take a random bacteria genome as a reference for 10 different bacteria genomes

In [ ]:
def test_reference(ref_length):
    import time
    dataset = "100bacteria"
    reference = get_genomes(dataset, sample=1)[0][:ref_length]
    for letter in ['A', 'C', 'T', 'G', 'N', '-']:
        if letter not in reference:
            reference += letter
    start_tree = time.time()
    compress_func = linear_ref_compress_genome(reference)
    end_tree = time.time()
    df = benchmark_functions([compress_regular, compress_func], dataset="10bacteria", sample=None)
    end_compress = time.time()
    return (df, end_tree - start_tree, end_compress - end_tree)

Test by length of reference

In [ ]:
performance_dict = {}
for k in [10, 100, 500, 1000, 1500]:
    performance_dict[k] = test_reference(k)

In [ ]:
def pct_compression(df):
    return ((df['compress_regular_new_len'] - df['compress_genome_new_len']) / df['compress_regular_new_len'] * 100).to_frame('pct_compression')

In [ ]:
import pandas as pd
performance_data = [[k, pct_compression(performance_dict[k][0]).mean()['pct_compression'], performance_dict[k][1], performance_dict[k][2]] for k in performance_dict.keys()]
performance_df = pd.DataFrame(performance_data, columns=["reference length", "pct compression (%)", "time to produce tree (s)", "time to run benchmark (s)"])

In [ ]:
performance_df.plot(x='reference length', y='pct compression (%)', title="% Compression Achieved by Reference Length")

In [ ]:
performance_df.plot(x='reference length', y='time to produce tree (s)', title="Time to Build Tree by Reference Length")

In [ ]:
performance_df.plot(x='reference length', y='time to run benchmark (s)', title="Time to Run Benchmark by Reference Length")

## Adding Heuristics

We know N and - present themselves heaps together so let's add some to this reference and see what happens

In [ ]:
def test_reference_heuristic(ref_length):
    import time
    dataset = "100bacteria"
    reference = get_genomes(dataset, sample=1)[0][:ref_length-150]
    # Add heuristic
    reference += "N" * 100
    reference += "-" * 50
    for letter in ['A', 'C', 'T', 'G', 'N', '-']:
        if letter not in reference:
            reference += letter
    start_tree = time.time()
    compress_func = linear_ref_compress_genome(reference)
    end_tree = time.time()
    df = benchmark_functions([compress_regular, compress_func], dataset="10bacteria", sample=None)
    end_compress = time.time()
    return (df, end_tree - start_tree, end_compress - end_tree)

In [ ]:
performance_H = test_reference_heuristic(1000)

In [ ]:
compare_df = pd.DataFrame([
    ["Heuristic", pct_compression(performance_H[0]).mean()['pct_compression']],
    ["No Heuristic", pct_compression(performance_dict[1000][0]).mean()['pct_compression']]
], columns=["Reference", "pct compression (%)"])
compare_df.plot.bar(x="Reference")

## Towards a universal reference...
Now, with the groundwork done, this becomes an optimization problem.

In the spirit of a computational biology class, we devise a genetic algorithm

Initial idea: Due to limitations of computational power. I'm thinking of doing a generative approach on references of length 500 of which we can construct trees relatively quickly. Then concatenate these together to form our final reference. The beauty about this tree approach is, it is done ONCE and then never again. So it can really be however long we like.

## Sequential Implementation

In [ ]:
from data_process import *

In [ ]:
# Define constants
N = 26 # number of children to consider
O = 2 # number of top old references to persist
K = 500 # length of references
GENERATIONS = 100
R = 2 # number randoms per generation

In [ ]:
def sample_random(ref_length):
    from random import randint
    complete = None
    while complete is None:
        try:
            k = randint(0,3)
            gen = get_genomes("100bacteria", sample=k)[k-1]
            complete = True
        except:
            pass
    start = randint(0,len(gen)-ref_length)
    sample = gen[start:start+ref_length]
    while any([b not in sample for b in ['A', 'C', 'T', 'G']]):
        start = randint(0,len(gen)-ref_length)
        sample = gen[start:start+ref_length]
    sample = sample[:-100] + ("N" * 50) + ("-" * 50)
    return sample

def generate_random_reference(ref_length):
    import random
    
    if random.random() > 0.9:
        return sample_random(ref_length)
    
    # Heuristic: N normally occur in groups
    # Heuristic: - normally occur individually or in groups
    prob_dict = {
        "A": 0.25,
        "C": 0.25,
        "T": 0.25,
        "G": 0.25,
    }
    N_len = 50 # Number of N to add to reference
    dash_len = 50 # Number of - to add to reference
    N_dash_len = N_len + dash_len
    
    # Sample from probabilities dict
    ref = "".join(random.choices(list(prob_dict.keys()), weights=prob_dict.values(), k=max(0, ref_length - N_dash_len)))
    # Where to insert N list
    insert_N_i = random.randint(0, len(ref))
    ref = ref[:insert_N_i] + ("N" * N_len) + ref[insert_N_i:]
    # Where to insert - list
    insert_N_i = random.randint(0, len(ref))
    ref = ref[:insert_N_i] + ("-" * dash_len) + ref[insert_N_i:]
    return ref

In [ ]:
def produce_offspring(ref1, ref2):
    import random
    bases = ['A', 'C', 'T', 'G', 'N', '-']
    assert(len(ref1) == len(ref2))
    split_i = random.randint(0, len(ref1))
    new = list(ref1[:split_i] + ref2[split_i:])
    # mutate
    for k in range(len(new)):
        if random.random() > 0.98:
            new[k] = bases[random.randint(0,5)]
    return "".join(new)

In [ ]:
def write_scores(score_dict, generation):
    with open(f"genetic/scores_out_{N}_{K}.txt", "a+") as f:
        f.write(f"Generation: {generation}\n")
        for r in score_dict.keys():
            f.write(f"{r},{str(score_dict[r])}\n")
        f.write("\n")
    with open(f"genetic/mean_pct_compression_{N}_{K}.txt", "a+") as f:
        avg_compression = sum(score_dict.values()) / len(score_dict.values())
        f.write(f"{generation},{str(avg_compression)}\n")

In [ ]:
"""# Generate initial references
reference_universe = [generate_random_reference(K) for _ in range(N)]
# Regular bitwise conversion
reg_df = benchmark_functions([compress_regular], dataset="10bacteria", sample=None)
for g in range(GENERATIONS):
    score_dict = {}
    # Test all references
    for r in reference_universe:
        compress_func = linear_ref_compress_genome(r)
        comp_df = benchmark_functions([compress_func], dataset="10bacteria", sample=None)
        score_dict[r] = ((reg_df['compress_regular_new_len'] - comp_df['compress_genome_new_len']) / reg_df['compress_regular_new_len'] * 100).to_frame('pct_compression').mean()['pct_compression']
    
    # Write scores
    write_scores(score_dict, g)
    
    # Get new references
    total_score = sum(score_dict.values())
    prob_list = [k/total_score for k in score_dict.values()]
    import random
    parents = random.choices(list(score_dict.keys()), weights=prob_list, k=(2*N))
    children = [produce_offspring(parents[i], parents[i+1]) for i in range(0, 2*N, 2)]
    children += [generate_random_reference(K) for _ in range(R)]
    reference_universe = children"""

### Paralellized

In [ ]:
from data_process import *

In [ ]:
def write_scores(score_list, ref_list, generation):
    with open(f"genetic/scores_out_{N}_{O}_{R}_{K}.txt", "a+") as f:
        f.write(f"Generation: {generation}\n")
        for r,s in zip(ref_list, score_list):
            f.write(f"{r},{str(s)}\n")
        f.write("\n")
    with open(f"genetic/mean_pct_compression_{N}_{O}_{R}_{K}.txt", "a+") as f:
        avg_compression = sum(score_list) / len(score_list)
        f.write(f"{generation},{str(avg_compression)}\n")

In [ ]:
import time
# Generate initial references
reference_universe = [generate_random_reference(K) for _ in range(N + R + O)]
# Regular bitwise conversion
reg_df = benchmark_functions([compress_regular], dataset="100bacteria", sample=20, check_atcg=True)
for g in range(GENERATIONS):
    start = time.time()
    print(f"Starting generation {g} with {len(reference_universe)} population")
    score_dict = {}
    # Test all references
    par_inputs = [(r, reg_df) for r in reference_universe]
    from multiprocessing import Pool
    pool = Pool(10)
    score_list = pool.map(test_ref, par_inputs)
    
    # Write scores
    write_scores(score_list, reference_universe, g)
    
    # Get new references
    total_score = sum(score_list)
    prob_list = [k/total_score for k in score_list]
    import random
    parents = random.choices(reference_universe, weights=prob_list, k=(2*N))
    children = [produce_offspring(parents[i], parents[i+1]) for i in range(0, 2*N, 2)]
    children += [generate_random_reference(K) for _ in range(R)]
    
    # add best old references to list
    best_prob = sorted(range(len(prob_list)), key=lambda i:prob_list[i])[::-1][:O]
    children += [reference_universe[k] for k in best_prob]
    
    reference_universe = children
    end = time.time()
    print(f"Ending, taken {end - start}s")

In [ ]:
with open('backup_refuni.txt', 'w+') as f:
    for x in reference_universe:
        f.write(f"{x}\n")